In [2]:
import cv2
import time
import numpy as np
import argparse

FLAGS = []

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--device", 
                        default="cpu")
    
    parser.add_argument('-v', '--video-path',
        default='sample_video.mp4')
    
    parser.add_argument('-w', '--weights',
        default='pose/mpi/pose_iter_160000.caffemodel')

    parser.add_argument('-cfg', '--config',
        default='pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt')

FLAGS, unparsed = parser.parse_known_args()



In [ ]:
MODE = "MPI"

if MODE is "MPI":
    nPoints = 8
    POSE_PAIRS = [1,2,3,4,5,6,7 ]

inWidth = 368
inHeight = 368
threshold = 0.1


input_source = FLAGS.video_path
cap = cv2.VideoCapture(input_source)
hasFrame, frame = cap.read()

vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))

net = cv2.dnn.readNetFromCaffe(FLAGS.config, FLAGS.weights)
if FLAGS.device == "cpu":
    net.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    print("Using CPU device")
elif FLAGS.device == "gpu":
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    print("Using GPU device")

while cv2.waitKey(1) < 0:
    t = time.time()
    hasFrame, frame = cap.read()
    frameCopy = np.copy(frame)
    if not hasFrame:
        cv2.waitKey()
        break

    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()

    H = output.shape[2]
    W = output.shape[3]
    # Empty list to store the detected keypoints
    points = []

    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        
        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        if prob > threshold : 
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append(None)

    

    # Draw Skeleton
    #Head – 0, Neck – 1, Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6, Left Wrist – 7,
#Right Hip – 8, Right Knee – 9, Right Ankle – 10, Left Hip – 11, Left Knee – 12, Left Ankle – 13, Chest – 14, Background – 15
    for pair in POSE_PAIRS:
        partA=POSE_PAIRS[0] #1
        partB=POSE_PAIRS[1] #2
        partC=POSE_PAIRS[2] #3
        partD=POSE_PAIRS[3] #4
        partE=POSE_PAIRS[4] #5
        partF=POSE_PAIRS[5] #6
        partG=POSE_PAIRS[6] #7
        
        #left arm
        cv2.circle(frame, points[partE], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.circle(frame, points[partF], 5, (0, 0, 255), thickness=1, lineType=cv2.FILLED)
        cv2.circle(frame, points[partG], 5, (0, 0, 255), thickness=1, lineType=cv2.FILLED)
        cv2.line(frame, points[partE], points[partF], (0, 255, 0), 3, lineType=cv2.LINE_AA)
        cv2.line(frame, points[partF], points[partG], (0, 255, 0), 3, lineType=cv2.LINE_AA)
        x1=cv2.norm(points[partE], points[partG])
        x2=cv2.norm(points[partF], points[partG])
    
        #right arm       
        cv2.circle(frame, points[partB], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.circle(frame, points[partC], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.circle(frame, points[partD], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.line(frame, points[partB], points[partC], (255, 0, 0), 3, lineType=cv2.LINE_AA)
        cv2.line(frame, points[partC], points[partD], (255, 0, 0), 3, lineType=cv2.LINE_AA)
        z1=cv2.norm(points[partB], points[partC])
        z2=cv2.norm(points[partC], points[partD])
     
        #shoulders to neck
        cv2.circle(frame, points[partA], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.line(frame, points[partA], points[partB], (0, 0, 255), 3, lineType=cv2.LINE_AA)
        left=cv2.norm(points[partA], points[partB])
        cv2.line(frame, points[partA], points[partE], (0, 0, 255), 3, lineType=cv2.LINE_AA)
        right=cv2.norm(points[partA], points[partE])
    #cv2.putText(frame, "Neck length = {:.2f} ".format(neck_leng), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    # cv2.putText(frame, "OpenPose using OpenCV", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 50, 0), 2, lineType=cv2.LINE_AA)
    # cv2.imshow('Output-Keypoints', frameCopy)
    
    print("left arm length: {:.3f}".format(x1+x2))
    print("left shoulder to neck + right shoulder to neck distance: {:.3f}".format(right+left))
    print("right arm length: {:.3f}".format(z1+z2))
    print('\n')
    cv2.imshow('Output-Skeleton', frame)
    vid_writer.write(frame)

vid_writer.release()

Using CPU device
left arm length: 92.000
left shoulder to neck + right shoulder to neck distance: 114.789
right arm length: 108.485


left arm length: 143.615
left shoulder to neck + right shoulder to neck distance: 116.949
right arm length: 96.540


left arm length: 204.749
left shoulder to neck + right shoulder to neck distance: 116.949
right arm length: 64.031


left arm length: 205.199
left shoulder to neck + right shoulder to neck distance: 123.471
right arm length: 72.498


left arm length: 206.547
left shoulder to neck + right shoulder to neck distance: 123.471
right arm length: 101.864


left arm length: 227.603
left shoulder to neck + right shoulder to neck distance: 112.830
right arm length: 133.665


left arm length: 199.885
left shoulder to neck + right shoulder to neck distance: 111.032
right arm length: 139.999


